In [5]:
# -*- coding: utf8 -*-


In [6]:
import  numpy as np
import pandas as pd

### 加载停用词词典

In [7]:
#加载停用词表
def loadStopDict(fileName):
    df = pd.read_csv(fileName,sep = ' ',encoding = 'utf-8', header=None)
    #print(df)
    df.columns = ['a']
    stopDict = (df['a'].tolist())
    return stopDict
#stopDict = loadStopDict("disStop.list")
stopDict = loadStopDict('../datasets/stopCat.list')
print(len(stopDict))

34191


In [8]:
print (stopDict[0:10])
if '一周' in stopDict:
    print ("1")

['一周', '一般情况', '一般状况', '上级', '下一阶段', '不', '不规律', '不详', '不适', '与']
1


### 计算词出现的文档数

In [9]:
#计算全局的idf
def calWordFre(fileName):

    word_idf = dict()
    word_dict = dict()
    allLines = 0
    for line in open(fileName):
        lineArr = line.strip().split(" ")
        lineList = lineArr
        lineSet = set(lineList)
        lineListNEW =list(lineSet)
        for i in range(len(lineListNEW)):
            #if word in dict , frequency +=1 ,else frequency =1
            disName = lineArr[i]
            if disName in stopDict:
                continue
            #word_freq[disName] = word_freq[disName]+1 if disName in word_freq else 1
            word_idf[disName] = word_idf[disName]+1 if disName in word_idf else 1
            
        allLines += 1
    i = 0
    
    for item in word_idf.keys():
        word_dict[item] = i
        i +=1
    print("word nums=",len(word_idf), i)
    return  word_idf, allLines, word_dict

In [10]:
filePath = "../rfModel/datasets/"
fileName = filePath + 'ADMIN.feature'
#labelName = filePath + '.label'
word_idf, allLines ,word_dict =  calWordFre(fileName)


word nums= 28484 28484


In [11]:
print("here")

here


In [12]:
#word nums= 62571 62571

In [13]:
#print(word_dict["住院期间"])
#print(len(word_dict), len(word_idf))

In [14]:
#f = open("tf-idf2.dict",'w')
#for item in word_dict:
#    f.write(item+"\n")
#f.flush()
#f.close()


### 计算tf*idf 

In [15]:
def  tfidf(word_freq, wordNum, word_idf, allLines, word):
    res = 0.0
    if( word in word_idf):
        tf = word_freq*1.0/wordNum
        idf = math.log(allLines*1.0 / (1 + word_idf[word]))
        res = tf*1.0*idf
        #print("tf=%f, idf=%f, res=%f" %(tf, idf, res))
    else:
        res = 0
    return res

### 统计一行输入的词频，与idf计算tf*idf

In [16]:
def  doWord2vecTrans(in_words,vecNum):
    vecSum=np.zeros(vecNum)
    word_freq = dict()
    wordNum = len(in_words)
    for word in in_words:
        word_freq[word] = word_freq[word]+1 if word in word_freq else 1
    #word_freq = word_freq*1.0/wordNum
    
    for word in in_words:
        #word = word.encode("utf8")
        #print("word=%s" %(word))
        #print("fre=%d num=%d word=%s dict=%d " %(word_freq[word], wordNum, word, word_dict[word]))
        if word in word_dict:
            vecSum[word_dict[word]] = tfidf(word_freq[word],wordNum,word_idf, allLines , word)
        #print("fre=%s" %(word_freq[word]))
    return vecSum

### 计算数据集的tf-idf向量


In [17]:
import math
import codecs
def  word2VecLoadTxt2vec(test_docs,output_file, vecNum):
    vecSumList = list()
    #output = open(output_file, "w")
    
    test_docs = [ x.strip().split() for x in codecs.open(test_docs, "r", "utf-8").readlines() ]
    #print test_docs[0]
    disVecList = list()

    for i in range(len(test_docs)):
        vecSum = doWord2vecTrans(test_docs[i], vecNum)
        vecSumList.append(vecSum)
        #output.write( ' '.join(str(x) for x in vecSum)+"\n")
    #output.flush()
    #output.close()
    return vecSumList

In [18]:
def loadDiseaseType2Vecotor( vecNum):
    #modelName="datasets/word2VecModel100cn-2.bin42"
    diseasetype_vecs = word2VecLoadTxt2vec("rfModel/datasets/trainADMIN.feature", "rfModel/datasets/trainADMIN.feature.vec" , vecNum)
    #diseasetype_vecs = loadVecFloat("rfModel/datasets/trainADMIN.feature.vec")
    
    diseasetype_vecsTest = word2VecLoadTxt2vec("rfModel/datasets/testADMIN.feature", "rfModel/datasets/testADMIN.feature.vec" , vecNum)
    #diseasetype_vecsTest = loadVecFloat("rfModel/datasets/testADMIN.feature.vec")
    #diseasetype_txt = loadTxt("datasets/diseasetype.mid")
    #print (len(diseasetype_vecs))
    return diseasetype_vecs,diseasetype_vecsTest

In [19]:
vecNum = len(word_dict)
diseasetype_vecs,diseasetype_vecsTest = loadDiseaseType2Vecotor(vecNum)

In [20]:
filePath1 = "../rfModel/datasets/train"
y_trainAll = list(open(filePath1+".label","r").readlines())
filePath2= "../rfModel/datasets/test"
y_testAll = list(open(filePath2+".label","r").readlines())

### 取前14类的数据

In [21]:
diseasetype_vecs  = diseasetype_vecs[0:86096]
y_trainAll = y_trainAll[0:86096]
diseasetype_vecsTest = diseasetype_vecsTest[0:21168]
y_testAll = y_testAll[0:21168]

In [22]:
import pickle
import os
cache_path = "vec_label.pik"
#if not os.path.exists(cache_path):
#    with open(cache_path, 'wb') as data_f:
#        pickle.dump((diseasetype_vecs,y_trainAll), data_f)



In [23]:
#diseasetype_vecs,y_trainAll =  pickle.load(open("vec_label.pik","rb"))
#print(len(diseasetype_vecs), len(y_trainAll))

### SVM L1和L2测试

In [26]:
from svmModel import *
svm_model_l1 =LinearSVCSearchL1My(diseasetype_vecs, y_trainAll, diseasetype_vecsTest, y_testAll) 

--------------Begin SVC Train!----------------
SVC Classifier trained in 25.207 seconds
LinearSVC(C=0.10000000000000001, class_weight=None, dual=False,
     fit_intercept=True, intercept_scaling=1, loss='squared_hinge',
     max_iter=1000, multi_class='ovr', penalty='l1', random_state=None,
     tol=0.0001, verbose=0)
this score: 0.72
             precision    recall  f1-score   support

        0 
       0.65      0.63      0.64      1577
        1 
       0.66      0.76      0.71      1589
       10 
       0.91      0.89      0.90      1599
       11 
       0.79      0.81      0.80      1145
       12 
       0.82      0.70      0.76      1254
       13 
       0.80      0.73      0.76      1290
        2 
       0.64      0.69      0.66      1591
        3 
       0.73      0.62      0.67      1583
        4 
       0.83      0.76      0.79      1598
        5 
       0.81      0.77      0.79      1599
        6 
       0.74      0.76      0.75      1591
        7 
       0.50    

In [25]:
from svmModel import *
svm_model_l2 =LinearSVCSearchL2My(diseasetype_vecs, y_trainAll, diseasetype_vecsTest, y_testAll)  

-------------Begin SVC Train!------------
SVC Classifier trained in 32.626 seconds
LinearSVC(C=0.10000000000000001, class_weight=None, dual=True,
     fit_intercept=True, intercept_scaling=1, loss='squared_hinge',
     max_iter=1000, multi_class='ovr', penalty='l2', random_state=None,
     tol=0.0001, verbose=0)
this score: 0.75
             precision    recall  f1-score   support

        0 
       0.68      0.67      0.67      1577
        1 
       0.70      0.80      0.74      1589
       10 
       0.93      0.90      0.92      1599
       11 
       0.80      0.82      0.81      1145
       12 
       0.84      0.73      0.79      1254
       13 
       0.82      0.75      0.78      1290
        2 
       0.69      0.72      0.70      1591
        3 
       0.77      0.67      0.72      1583
        4 
       0.85      0.80      0.82      1598
        5 
       0.82      0.85      0.83      1599
        6 
       0.79      0.79      0.79      1591
        7 
       0.63      0.49

In [23]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV
from time import time
def rfModel():
    # 设置参数
    '''  
    param_dist = {"max_depth": [3, 4],
              "max_features": [1,5,11],
              "min_samples_split": [1,5,11],
              "min_samples_leaf": [1,5,11],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}
    # 运行随机搜索 RandomizedSearch
    n_iter_search = 20
    # 元分类器
    meta_clf = RandomForestClassifier(n_estimators=16)
    rs_clf = RandomizedSearchCV(meta_clf, param_distributions=param_dist,
                                    n_iter=n_iter_search)
    ''' 
    param_dist = {"max_depth": [3, 4],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}
    # 运行随机搜索 RandomizedSearch
    n_iter_search = 20
    # 元分类器
    meta_clf = RandomForestClassifier(n_estimators=16)
    rs_clf = GridSearchCV(meta_clf, param_dist)
    start = time()
    rs_clf.fit(diseasetype_vecs, y_trainAll)
    print("RandomizedSearchCV took %.2f seconds for %d candidates"
          " parameter settings." % ((time() - start), n_iter_search))
    return rs_clf
#rs_clf = rfModel()

### rf模型训练和测试

In [24]:
#from svmModel import *
#rfModelTrain(diseasetype_vecs,y_trainAll,diseasetype_vecsTest,y_testAll )

### svm模型训练和测试

In [15]:
from svmModel import *
import numpy as np

[0.10000000000000001, 0.20000000000000001, 0.30000000000000004, 0.40000000000000002, 0.5, 0.59999999999999998, 0.70000000000000007, 0.80000000000000004, 0.90000000000000002, 1.0, 1.1000000000000001, 1.2000000000000002, 1.3000000000000003, 1.4000000000000001, 1.5000000000000002, 1.6000000000000001, 1.7000000000000002, 1.8000000000000003, 1.9000000000000001, 2.0, 2.1000000000000001, 2.2000000000000002, 2.3000000000000003, 2.4000000000000004, 2.5000000000000004, 2.6000000000000001, 2.7000000000000002, 2.8000000000000003, 2.9000000000000004, 3.0000000000000004]
[10, 100, 300, 500, 800, 1000]
[0.10000000000000001, 0.20000000000000001, 0.30000000000000004, 0.40000000000000002, 0.5, 0.59999999999999998, 0.70000000000000007, 0.80000000000000004, 0.90000000000000002, 1.0, 1.1000000000000001, 1.2000000000000002, 1.3000000000000003, 1.4000000000000001, 1.5000000000000002, 1.6000000000000001, 1.7000000000000002, 1.8000000000000003, 1.9000000000000001, 2.0, 2.1000000000000001, 2.2000000000000002, 2

In [21]:
svm_model_l1 =LinearSVCSearchL1My(diseasetype_vecs,y_trainAll,diseasetype_vecsTest,y_testAll) 

--------------Begin SVC Train!----------------
SVC Classifier trained in 30.477 seconds
LinearSVC(C=0.10000000000000001, class_weight=None, dual=False,
     fit_intercept=True, intercept_scaling=1, loss='squared_hinge',
     max_iter=1000, multi_class='ovr', penalty='l1', random_state=None,
     tol=0.0001, verbose=0)
this score: 0.68
             precision    recall  f1-score   support

        0 
       0.60      0.63      0.61      1577
        1 
       0.62      0.76      0.68      1589
       10 
       0.89      0.88      0.89      1599
       11 
       0.78      0.81      0.80      1145
       12 
       0.79      0.70      0.75      1254
       13 
       0.78      0.73      0.75      1290
       14 
       0.57      0.29      0.38      1596
        2 
       0.60      0.69      0.64      1591
        3 
       0.69      0.62      0.65      1583
        4 
       0.79      0.75      0.77      1598
        5 
       0.79      0.78      0.78      1599
        6 
       0.72    

In [20]:
svm_model_l2 =LinearSVCSearchL2My(diseasetype_vecs,y_trainAll,diseasetype_vecsTest,y_testAll) 

-------------Begin SVC Train!------------
SVC Classifier trained in 37.1 seconds
LinearSVC(C=0.10000000000000001, class_weight=None, dual=True,
     fit_intercept=True, intercept_scaling=1, loss='squared_hinge',
     max_iter=1000, multi_class='ovr', penalty='l2', random_state=None,
     tol=0.0001, verbose=0)
this score: 0.72
             precision    recall  f1-score   support

        0 
       0.64      0.66      0.65      1577
        1 
       0.67      0.80      0.73      1589
       10 
       0.91      0.90      0.91      1599
       11 
       0.78      0.82      0.80      1145
       12 
       0.82      0.74      0.78      1254
       13 
       0.80      0.75      0.77      1290
       14 
       0.77      0.34      0.47      1596
        2 
       0.64      0.72      0.68      1591
        3 
       0.74      0.67      0.70      1583
        4 
       0.82      0.79      0.80      1598
        5 
       0.79      0.85      0.82      1599
        6 
       0.76      0.79  

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
#pca = PCA(n_components='mle')
#pca = PCA(n_components=0.95)
#pca.fit(diseasetype_vecs)
#print (pca.explained_variance_ratio_)
#print (pca.explained_variance_)
#print (pca.n_components_)

In [32]:
from time import time
def svcL1(punish, max_iter,multi_class, featuresList, labelList):
    from sklearn.svm import LinearSVC
    t0 = time()
    print ("Begin SVC Train!")
    
    svc = LinearSVC(penalty = "l1", C=punish, dual=False, max_iter = max_iter, multi_class = multi_class)
                #(penalty, loss='squared_hinge', dual=True, tol=0.0001, C=punish, multi_class='ovr' , 
                    #fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0, random_state=None, max_iter=5000)
    svc_model = svc.fit(featuresList, labelList) 
    tt = time() - t0
    print ("SVC Classifier trained in {} seconds".format(round(tt,3)))
    return svc_model

In [33]:
import numpy as np
l = range(1,5)
print(l)
s = list(np.arange(0.1,3,0.1))
print(s)
t = list([10,100,300,500,800,1000])
print(t)
ss = s+ t
print(ss)

range(1, 5)
[0.10000000000000001, 0.20000000000000001, 0.30000000000000004, 0.40000000000000002, 0.5, 0.59999999999999998, 0.70000000000000007, 0.80000000000000004, 0.90000000000000002, 1.0, 1.1000000000000001, 1.2000000000000002, 1.3000000000000003, 1.4000000000000001, 1.5000000000000002, 1.6000000000000001, 1.7000000000000002, 1.8000000000000003, 1.9000000000000001, 2.0, 2.1000000000000001, 2.2000000000000002, 2.3000000000000003, 2.4000000000000004, 2.5000000000000004, 2.6000000000000001, 2.7000000000000002, 2.8000000000000003, 2.9000000000000004]
[10, 100, 300, 500, 800, 1000]
[0.10000000000000001, 0.20000000000000001, 0.30000000000000004, 0.40000000000000002, 0.5, 0.59999999999999998, 0.70000000000000007, 0.80000000000000004, 0.90000000000000002, 1.0, 1.1000000000000001, 1.2000000000000002, 1.3000000000000003, 1.4000000000000001, 1.5000000000000002, 1.6000000000000001, 1.7000000000000002, 1.8000000000000003, 1.9000000000000001, 2.0, 2.1000000000000001, 2.2000000000000002, 2.3000000

In [34]:
print(len(diseasetype_vecs))

92496


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.externals import joblib
from sklearn.metrics import classification_report
def LinearSVCSearchL1My():
    #param_grid={"C": [0.001, 0.01, 0.1, 1.0, 5.0,10.0], "penalty": "l2", "multi_class":["ovr","crammer_singer"],"max_iter":[500,1000,2000]}
    #L2 gridsearch
    #param_grid=[{"C": [0.001, 0.01, 0.1, 1.0, 5.0,10.0,100,500,100], "penalty": ["l2"], "multi_class":["ovr","crammer_singer"],"max_iter":[500,1000,2000]}]
    #L1 gridsearch
    param_grid=[{"C": ss, "penalty": ["l1"],"multi_class":["ovr","crammer_singer"],"max_iter":[500,1000,2000]}]
    best = 0
    bestModel = "svmModels/svm_l1_train_model.m"
    for c in ss:
        for max_iter in [1000,500,2000]:
            for multi_class in ["ovr","crammer_singer"]:
            #for multi_class in ["crammer_singer"]:
                svc_model = svcL1(c, max_iter,multi_class, diseasetype_vecs, y_trainAll)
                y_true, y_pred = y_testAll, svc_model.predict(diseasetype_vecsTest)
                #print(y_true[0:10], y_pred[0:10])
                res = classification_report(y_true,y_pred)
                resArr = (res.strip().split("\n"))
                score = float(resArr[len(resArr)-1].strip().split(" ")[21])
                print(svc_model)
                print("this score:", score)
                print(res)
                if(score > best):
                    best = score
                    print("===========================best fscore:",best)
                    print(svc_model)
                    print(res)
                    joblib.dump(svc_model, bestModel+"_"+str(c)+"_"+str(max_iter)+"_"+str(multi_class))
                

    return clf
clf = LinearSVCSearchL1My()

Begin SVC Train!
SVC Classifier trained in 31.342 seconds
LinearSVC(C=0.10000000000000001, class_weight=None, dual=False,
     fit_intercept=True, intercept_scaling=1, loss='squared_hinge',
     max_iter=1000, multi_class='ovr', penalty='l1', random_state=None,
     tol=0.0001, verbose=0)
this score: 0.68
             precision    recall  f1-score   support

        0 
       0.60      0.63      0.61      1577
        1 
       0.62      0.76      0.68      1589
       10 
       0.89      0.88      0.89      1599
       11 
       0.78      0.81      0.80      1145
       12 
       0.79      0.70      0.75      1254
       13 
       0.78      0.73      0.75      1290
       14 
       0.57      0.29      0.38      1596
        2 
       0.60      0.69      0.64      1591
        3 
       0.69      0.62      0.65      1583
        4 
       0.79      0.75      0.77      1598
        5 
       0.79      0.78      0.78      1599
        6 
       0.72      0.76      0.74      1591
   

KeyboardInterrupt: 

In [ ]:
from sklearn import datasets
from sklearn import svm, grid_search, datasets
from sklearn.metrics import classification_report
from sklearn.svm import SVC

from sklearn.linear_model import Ridge                   #L2正则化
from sklearn.linear_model import Lasso  

from time import time

# In[39]:

import numpy as np

s = list(np.arange(0.8,1.4,0.1))
print(s)
t = list([10,100,300,500,800])
print(t)
ss = s+ t
print(ss)


# In[80]:


def svcL2(punish, max_iter,multi_class, featuresList, labelList):
    from sklearn.svm import LinearSVC
    t0 = time()
    print ("Begin SVC Train!")
    
    svc = LinearSVC(penalty = "l2", C=punish, max_iter = max_iter, multi_class = multi_class)
                #(penalty, loss='squared_hinge', dual=True, tol=0.0001, C=punish, multi_class='ovr' , 
                    #fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0, random_state=None, max_iter=5000)
    svc_model = svc.fit(featuresList, labelList) 
    tt = time() - t0
    print ("SVC Classifier trained in {} seconds".format(round(tt,3)))
    return svc_model

from sklearn.svm import LinearSVC
from sklearn.externals import joblib
def LinearSVCSearchL2My():
    #param_grid={"C": [0.001, 0.01, 0.1, 1.0, 5.0,10.0], "penalty": "l2", "multi_class":["ovr","crammer_singer"],"max_iter":[500,1000,2000]}
    #L2 gridsearch
    #param_grid=[{"C": [0.001, 0.01, 0.1, 1.0, 5.0,10.0,100,500,100], "penalty": ["l2"], "multi_class":["ovr","crammer_singer"],"max_iter":[500,1000,2000]}]
    #L1 gridsearch
    param_grid=[{"C": ss, "penalty": ["l2"],"multi_class":["ovr","crammer_singer"],"max_iter":[500,1000,2000]}]
    best = 0
    bestModel = "svmModels/svm_l1_train_model.m"
    for c in ss:
        for max_iter in [1000,500,2000]:
            for multi_class in ['ovr',"crammer_singer"]:
                svc_model = svcL2(c, max_iter,multi_class, diseasetype_vecs, y_trainAll)
                y_true, y_pred = y_testAll, svc_model.predict(diseasetype_vecsTest)
                #print(y_true[0:10], y_pred[0:10])
                res = classification_report(y_true,y_pred)
                resArr = (res.strip().split("\n"))
                score = float(resArr[len(resArr)-1].strip().split(" ")[21])
                print(svc_model)
                print("this score:", score)
                print(res)
                if(score > best):
                    best = score
                    print("===========================best fscore:",best)
                    print(svc_model)
                    print(res)
                    joblib.dump(svc_model, bestModel+"_"+str(c)+"_"+str(max_iter)+"_"+str(multi_class))
                

    return clf
clf = LinearSVCSearchL2My()

[0.80000000000000004, 0.90000000000000002, 1.0, 1.1000000000000001, 1.2, 1.2999999999999998]
[10, 100, 300, 500, 800]
[0.80000000000000004, 0.90000000000000002, 1.0, 1.1000000000000001, 1.2, 1.2999999999999998, 10, 100, 300, 500, 800]
Begin SVC Train!


In [ ]:
from sklearn.svm import LinearSVC
def LinearL2SVCSearch():
    #param_grid={"C": [0.001, 0.01, 0.1, 1.0, 5.0,10.0], "penalty": "l2", "multi_class":["ovr","crammer_singer"],"max_iter":[500,1000,2000]}
    param_grid=[{"C": ss, "penalty": ["l2"], "multi_class":["crammer_singer"],"max_iter":[500,1000,2000]}]
    #tuned_parameters = [{'kernel': ['linear','rbf','poly'], 'C': [1e-3,1e-2,1e-1,1, 10,100, 1000], 'decision_function_shape':['ovo', 'ovr']}]

    clf = grid_search.GridSearchCV(LinearSVC(), param_grid  )
    clf.fit(vecList, labelList)
    # 再调用 clf.best_params_ 就能直接得到最好的参数搭配结果
    print(clf.best_params_)

    y_true, y_pred = labelListTest, clf.predict(vecListTest)

    # 打印在测试集上的预测结果与真实值的分数
    print(classification_report(y_true, y_pred))
    print(clf.best_estimator_)
    print(clf.best_params_)
    return clf
'''
clf = LinearL2SVCSearch()
print(clf.grid_scores_)
best_svm = clf.best_estimator_
best_svm
best_svm.fit(diseasetype_vecs, y_trainAll)
from sklearn.externals import joblib
joblib.dump(best_svm, "svmModels/svm_l1_train_model_tfidf.m")
pre = best_svm.predict(np.array(diseasetype_vecsTest, dtype='f'))
print(set(pre))
print(set(y_testAll))
print(classification_report(np.array(y_testAll,dtype="f"),np.array(best_svm.predict(diseasetype_vecsTest), dtype='f')))
'''

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn import svm, grid_search
punishList = punishList = [0.1,1,10]#np.arange(0.1,1.0,0.1)
bestSVCModel = None
bestTestAccuracy = 0
accuracyList = list()
'''
for punish in punishList:
    svc_model = doSVCTrainL2(np.array(diseasetype_vecs,dtype= "f"),np.array( y_trainAll,dtype="f"), punish)
    print (svc_model)
    #test_accuracy = doSVCTest(np.array(vecListTest,dtype= "f"),np.array(labelList,dtype="f"), svc_model)
    print(classification_report(np.array(y_testAll,dtype="f"),svc_model.predict(diseasetype_vecsTest)))

for punish in punishList:
    svc_model = doSVCTrainL1(np.array(diseasetype_vecs,dtype= "f"),np.array( y_trainAll,dtype="f"), punish)
    print (svc_model)
    #test_accuracy = doSVCTest(np.array(vecListTest,dtype= "f"),np.array(labelList,dtype="f"), svc_model)
    print(classification_report(np.array(y_testAll,dtype="f"),svc_model.predict(diseasetype_vecsTest)))
'''    


In [ ]:
#!jupyter nbconvert --to script SVMModelTrainInTFIDF.ipynb